# Submitting Your Program to *Aquila*

When you feel you've succesfully figured out the geometry and pulse sequence you'd like to try on hardware, you're ready to submit things to *Aquila*!

This notebook will serve as a handy guide on how to do just that with an example MIS program taken straight from the [MIS Bloqade Tutorial](https://queracomputing.github.io/bloqade-python-examples/latest/examples/example-5-MIS-UDG/) (The example has been slightly modified from its original form to keep costs as low as possible)

## Amazon Braket Hybrid Jobs

You'll submit your Bloqade program as an Amazon Braket Hybrid Job. In this configuration the Bloqade code actually gets submitted to and executed on an AWS EC2 instance (a cloud computing resource) that has the ability to then submit your tasks to *Aquila*. 

You can then monitor the status of your program in the cloud and when it's complete retrieve the results.

## Prerequisites

Before submitting things to *Aquila* for this hackathon you'll need to have the 
following things ready:

### Amazon Braket SDK

You won't be using this to build your program (that's what Bloqade is for!) but it does provide the tools needed to create hybrid jobs and retrive your results. You can install it via `pip install amazon-braket-sdk`

### Credentials

Your team should have a QuEra-provided `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` strings. You'll need these to authenticate with your AWS Braket accounts to both submit your program as well as monitor its status on the queue.

### `requirements.txt`

In order to run your Bloqade program on the cloud, AWS Braket needs some knowledge of the software dependencies required. In this case all you need is Bloqade so create a blank text file with the name `requirements.txt` and put the text `bloqade` in all lower case. Make sure you put this somewhere easily accessible as you'll have to tell the Braket SDK where it's located on your system.



## The Program

Let's construct a Bloqade program for solving the MIS:

In [3]:
from bloqade.atom_arrangement import Square
import numpy as np

# setting the seed
rng = np.random.default_rng(1234)

durations = [0.3, 1.6, 0.3]

mis_udg_program = (
    Square(15, lattice_spacing=5.0)
    .apply_defect_density(0.3, rng=rng)
    .rydberg.rabi.amplitude.uniform.piecewise_linear(durations, [0.0, 15.0, 15.0, 0.0])
    .detuning.uniform.piecewise_linear(
        durations, [-30, -30, "final_detuning", "final_detuning"]
    )
)

mis_udg_job = mis_udg_program.assign(final_detuning=80)

## Submitting the Program

Now let's import what we need from the AWS Braket SDK

In [4]:
from braket.devices import Devices
from braket.jobs import hybrid_job

We now need to give the Braket SDK our credentials. The easiest way to do this is to set the proper environment variables. In Jupyter you do it like so: 

In [5]:
%env AWS_ACCESS_KEY_ID=YOUR_ACCESS_KEY_ID
%env AWS_SECRET_ACCESS_KEY=YOUR_AWS_SECRET_ACCESS_KEY
%env AWS_DEFAULT_REGION=us-east-1

env: AWS_ACCESS_KEY_ID=YOUR_ACCESS_KEY_ID
env: AWS_SECRET_ACCESS_KEY=YOUR_AWS_SECRET_ACCESS_KEY
env: AWS_DEFAULT_REGION=us-east-1


NOTE: Make sure you have that last environment variable set to `us-east-1`! Otherwise your program will not submit. 

Now we wrap our program into a function and then decorate the function using the `hybrid_job` decorator. This is where you provide a path to your `requirements.txt` and specify the device the program will submit tasks to.

Notice that we are calling `.report` on our program immediately after it runs and returning it from the function. 

This report is what you will retrieve when the job is done running on the cloud!

In [ ]:
@hybrid_job(device=Devices.QuEra.Aquila, dependencies="requirements.txt")
def your_job():
    return mis_udg_job.braket.aquila().run(100).report()

All we have to do now is call the function and save its result. This will submit your program to the cloud to run and give you an `AwsQuantumJob` that you can use to track the status of your job and pull the actual Bloqade results from.

In [ ]:
cloud_job = your_job()

## Tracking and Getting Results From Your Program

We can see what the state of your job is (queued, running, completed, etc.) via: 

In [ ]:
cloud_job.state()

You can see the position of your job on the queue via: 

In [ ]:
cloud_job.queue_position()

As good practice, you should save the ARN (Amazon Resource Number) of your Job so if your notebook runs into problems you can "rebuild" the `AwsQuantumJob` object and not lose your progress:

In [ ]:
job_arn = cloud_job.arn

from braket.aws.aws_quantum_job import AwsQuantumJob
duplicate_cloud_job = AwsQuantumJob(job_arn)

When you see `cloud_job.state()` is "COMPLETED" you can now retrieve your results!

In [ ]:
result_dict = cloud_job.result()
report = result_dict["result"]

Report is a Bloqade object that lets you visualize results, get bitstrings and counts of bitstrings, etc.

In [6]:
#report.show()
#report.bitstrings()
#report.counts()
#report.rydberg_densities()
#etc...

You can check out the [Bloqade tutorials](https://queracomputing.github.io/bloqade-python-examples/latest/) for some inspiration on how to do more advanced results analysis too! 